# Single-copy state characterization

* It loads reconstructed single-copy (three-qubit) data from  `data/reconstructed_three_qubit_states.h5` and calculates the following figures of merit:
    * purity
    * fidelity
    * PPT criterion 
* The analysis is performed on Monte-Carlo samples to estimate the statistical uncertainty and mean value.
* See `reconstruct.ipynb` to see how the Monte-Carlo samples are acquired.

## Prerequisites and definitions

In [1]:
import itertools
import dataclasses
from enum import IntEnum
import numpy as np
import scipy
import matplotlib.pyplot as plt
import h5py
from scipy.optimize import minimize
import departed
from bisep_checker import (
    check_bisep,
    partitions,
    EPSILON,
    dagger,
    ketbra,
    kron,
    theo_rhos_3q,
    BinKet,
    HLO,
    HHI,
    LO,
    HI,
    CLO,
    CHI,
    purity,
)
from IPython.display import Math, Markdown


def psd_projection(r: np.ndarray) -> np.ndarray:
    # force Hermiteanity
    hr = (r + r.T.conj()) * 0.5
    # do spectral decomposition
    evals, evecs = np.linalg.eigh(hr)
    # clip negative eigenvalues, 1e-16 is numerical epsilon
    devals = np.diag(np.clip(evals.real, 1e-16, None))
    # project to PSD matrix
    rp = evecs @ devals @ evecs.T.conj()
    # renorm and return
    return rp / np.trace(rp.real)

def purity(rho):
    return np.trace(rho @ rho).real


def gramm_schmidt(X, row_vecs=False, norm=True):
    """
    Vectorized Gramm-Schmidt orthogonalization.
    Creates an orthonormal system of vectors spanning the same vector space
    which is spanned by vectors in matrix X.

    Args:
        X: matrix of vectors
        row_vecs: are vectors store as line vectors? (if not, then use column vectors)
        norm: normalize vector to unit size
    Returns:
        Y: matrix of orthogonalized vectors
    Source: https://gist.github.com/iizukak/1287876#gistcomment-1348649
    """
    if not row_vecs:
        X = X.T
    Y = X[0:1, :].copy()
    for i in range(1, X.shape[0]):
        proj = np.diag((X[i, :].dot(Y.T.conj()) / np.linalg.norm(Y, axis=1) ** 2).flat).dot(Y)
        Y = np.vstack((Y, X[i, :] - proj.sum(0)))
    if norm:
        Y = np.diag(1 / np.linalg.norm(Y, axis=1)).dot(Y)
    if row_vecs:
        return Y
    else:
        return Y.T

def sqrtm(M):
    """
    Matrix square root of density matrix M.
    Calculation is based on eigen-decomposition.    
    """
    Di, Rot = np.linalg.eig(M)
    rank = np.sum((np.abs(Di) > 2*np.finfo(float).eps))
    Di = np.sqrt(Di)
    Di[np.isnan(Di)] = 0
    Di = np.diag(Di)
    if (rank == M.shape[0]):
        #Full rank => Hermitean transposition is actually inversion
        RotInv = Rot.T.conjugate()
    elif rank == 1:
        #Rank 1 => The state is pure and the matrix is it's own square-root.
        return M
    else:
        #(1 < Rank < dimension) => at least one eigenvalue is zero, orthogonalize found unitary
        #in order to perform Hermitean inversion of the rotation matrix
        #If this was not the case, zero eigenvalue can correspond to 
        #arbitrary vector which would destroy unitarity of Rot matrix.
        RotGs = gramm_schmidt(Rot, False, True)
        RotInv = RotGs.T.conjugate()
    N = np.dot(np.dot(Rot, Di), RotInv)
    return N

def fidelity(A, B):
    """
    Fidelity between density matrices A, B.
    Accepts both mixed. If A or B is pure, consider using Overlap().
    """
    #A0 = sqrtm(A)
    Ax = A/np.trace(A)
    Bx = B/np.trace(B)
    A0 = scipy.linalg.sqrtm(Ax)
    #use scipy to do sqrtm, when it fails (for singular matrices), try KetSugars sqrtm method
    if np.any(np.isnan(A0)):
        A0 = sqrtm(Ax)    
    A1 = (np.dot(np.dot(A0, Bx), A0))
    A2 = scipy.linalg.sqrtm(A1)
    if np.any(np.isnan(A2)):
        A2 = sqrtm(A1)       
    return np.abs(A2.trace())**2

# check PPT
def ppt(rho, pt : partitions):
    if pt == partitions.A_BC:
        mask1 = [1,0,0]
    elif pt == partitions.AC_B:
        mask1 = [0,1,0]
    elif pt == partitions.AB_C:
        mask1 = [0,0,1]        
    else:
        print("Ooops!")
        raise
    
    rhopt = departed.ptranspose(rho, [2]*3, mask1)
    mineig = np.min(np.linalg.eigvalsh(rhopt))
    return mineig
# check PPT
def ppt(rho, pt : partitions):
    if pt == partitions.A_BC:
        mask1 = [1,0,0]
    elif pt == partitions.AC_B:
        mask1 = [0,1,0]
    elif pt == partitions.AB_C:
        mask1 = [0,0,1]        
    else:
        print("Ooops!")
        raise
    
    rhopt = departed.ptranspose(rho, [2]*3, mask1)
    mineig = np.min(np.linalg.eigvalsh(rhopt))
    return mineig



@dataclasses.dataclass
class aggregated_result():
    mean : float
    median : float
    std : float
    q158 : float
    q842 : float

    @classmethod
    def from_array(cls, arr):
        return cls(
            mean = np.mean(arr),
            median = np.median(arr),
            std = np.std(arr),
            q158 = np.quantile(arr, q=0.158),
            q842 = np.quantile(arr, q=0.842)
        )

    def format_to_error_q(self, median=None):
        if median is None:
            mean = self.median
        else:
            mean = median
        lo = self.q158
        hi = self.q842
        dlo = mean - lo
        dhi = hi - mean
        significant_order = int(min(np.floor(np.log10(abs(dlo))), np.floor(np.log10(abs(dhi)))))
        exponent = 10**(-significant_order)
        rounded_mean = int(round(exponent*mean))
        rounded_dlo = int(round(exponent*dlo))
        rounded_dhi = int(round(exponent*dhi))
        repre = f'$({rounded_mean:d}^{{ +{rounded_dhi}}}_{{-{rounded_dlo} }}) \cdot 10^{{ {significant_order:d} }}$'
        return repre
    
    def format_to_error_std(self, median=None):
        if median is None:
            mean = self.median
        else:
            mean = median
        std = self.std
        significant_order = int(np.floor(np.log10(abs(std))))
        exponent = 10**(-significant_order)
        rounded_mean = int(round(exponent*mean))
        rounded_std = int(round(exponent*std))        
        repre = f'$({rounded_mean:d}\pm {rounded_std}) \cdot 10^{{ {significant_order:d} }}$'
        return repre
        
    def display_in_notebook_q(self, median=None):
        latex = self.format_to_error_q(median=median)
        latex_code_md = f'```latex\n{latex}\n```'
        display(Math(latex))
        display(Markdown(latex_code_md))

    def display_in_notebook_std(self, median=None):
        latex = self.format_to_error_std(median=median)
        latex_code_md = f'```latex\n{latex}\n```'
        display(Math(latex))
        display(Markdown(latex_code_md))

In [7]:
# Load ML-reconstructed single-copy constituent states as a single numpy array
with h5py.File("data/reconstructed_three_qubit_states.h5") as h5f:
    rhos_single_copy_024 = np.array(h5f["rho_024"])
    rhos_single_copy_531 = np.array(h5f["rho_531"])

# Load tomogram files, just to pick the saved state order
with h5py.File('data/three_qubit_tomograms.h5') as h5f:
    state_order = list(h5f["ions_024"].attrs.get("state_order", []))

rhos_single_copy_024 = np.array([psd_projection(_r) for _r in rhos_single_copy_024])
rhos_single_copy_531 = np.array([psd_projection(_r) for _r in rhos_single_copy_531])

In [8]:
#mixing factor
q = 0.06
wghts = [(1-q)/8]*8 + [q/2]*2

#mix together the constituent states
rho_exp_024 = sum([w*r for r, w in zip(rhos_single_copy_024, wghts)])
rho_exp_531 = sum([w*r for r, w in zip(rhos_single_copy_531, wghts)])
rho_theo = sum([w*theo_rhos_3q[key] for key, w in zip(theo_rhos_3q, wghts)])

In [10]:
with h5py.File("data/reconstructed_three_qubit_states.h5") as h5f:
    rhos_single_copy_024 = np.array(h5f["rhos_024"])
    rhos_single_copy_531 = np.array(h5f["rhos_531"])
    rho_seed_024 = np.array(h5f["rho_024"])
    rho_seed_531 = np.array(h5f["rho_531"])
    
rho_exp_024 = sum([w*r for r, w in zip(rho_seed_024, wghts)])
rho_exp_531 = sum([w*r for r, w in zip(rho_seed_531, wghts)])

wghts = np.array(wghts)
rhos_single_copy_024_summed = np.sum(rhos_single_copy_024 * wghts.reshape((1,10,1,1)), axis=1)
rhos_single_copy_531_summed = np.sum(rhos_single_copy_531 * wghts.reshape((1,10,1,1)), axis=1)

## Print purity 

In [11]:
#Assess purity


#Theoretical
P_theo = np.trace(rho_theo @ rho_theo).real
print('theo', P_theo)

#Seed 1
print('seed1', purity(rho_exp_024))

#Seed 2
print('seed2', purity(rho_exp_531))

#Monte carlo samples
purs_024 = np.array([purity(r) for r in rhos_single_copy_024_summed])
pur_stat_024 = aggregated_result.from_array(purs_024)
print(pur_stat_024.mean, pur_stat_024.std)
pur_stat_024.display_in_notebook_q(purity(rho_exp_024))

#Monte carlo samples
purs_531 = np.array([purity(r) for r in rhos_single_copy_531_summed])
pur_stat_531 = aggregated_result.from_array(purs_531)
print(pur_stat_531.mean, pur_stat_531.std)
pur_stat_531.display_in_notebook_q(purity(rho_exp_531))


theo 0.27792499999999987
seed1 0.25228982944625555
seed2 0.2544540800405938
0.25264928756447874 0.00218382332458198


<IPython.core.display.Math object>

```latex
$(252^{ +2}_{-2 }) \cdot 10^{ -3 }$
```

0.25436283209519633 0.0024172749089337972


<IPython.core.display.Math object>

```latex
$(254^{ +2}_{-2 }) \cdot 10^{ -3 }$
```

## Assess fidelity

In [12]:
fidelity(rho_exp_024, rho_exp_531)

0.9862166323710658

In [13]:
## fidelity

#Seed 1
f1 = fidelity(rho_exp_024, rho_theo)
print('seed1', f1)

#Seed 2
f2 = fidelity(rho_exp_531, rho_theo)
print('seed2', f2)


fids_024 = np.array([fidelity(r, rho_theo) for r in rhos_single_copy_024_summed])
fids_531 = np.array([fidelity(r, rho_theo) for r in rhos_single_copy_531_summed])

#Monte carlo samples
print("MC1")
fids_stat_024 = aggregated_result.from_array(fids_024)
print(fids_stat_024.mean, fids_stat_024.std)
fids_stat_024.display_in_notebook_q(f1)
fids_stat_024.display_in_notebook_q()

print("MC2")
fids_stat_531 = aggregated_result.from_array(fids_531)
print(fids_stat_531.mean, fids_stat_531.std)
fids_stat_531.display_in_notebook_q(f2)
fids_stat_531.display_in_notebook_q()


seed1 0.9746040710716742
seed2 0.9870302471899676
MC1
0.9750893717197558 0.0015118028527687241


<IPython.core.display.Math object>

```latex
$(9746^{ +19}_{-9 }) \cdot 10^{ -4 }$
```

<IPython.core.display.Math object>

```latex
$(975^{ +1}_{-2 }) \cdot 10^{ -3 }$
```

MC2
0.9857418731835019 0.0014005107383316831


<IPython.core.display.Math object>

```latex
$(9870^{ +1}_{-24 }) \cdot 10^{ -4 }$
```

<IPython.core.display.Math object>

```latex
$(986^{ +1}_{-1 }) \cdot 10^{ -3 }$
```

In [14]:
#Predicted overall fidelity

rho_theo_both = np.kron(rho_theo, rho_theo)
rho_exp_both = np.kron(rho_exp_531, rho_exp_531)

fboth = fidelity(rho_theo_both, rho_exp_both)
print('F_2 = ', fboth)

fids_both = np.array([
    fidelity(np.kron(r, s), rho_theo_both) 
    for r, s in zip(rhos_single_copy_024_summed, rhos_single_copy_531_summed)])

print("Extrapolated")
fids_stat_both = aggregated_result.from_array(fids_both)
print(fids_stat_both.mean, fids_stat_both.std)
# fids_stat_both.display_in_notebook_q(fboth)
fids_stat_both.display_in_notebook_q()

F_2 =  0.9742287214190596
Extrapolated
0.9611865491293008 0.002074658979692732


<IPython.core.display.Math object>

```latex
$(962^{ +1}_{-2 }) \cdot 10^{ -3 }$
```

## PPT

In [15]:
for pt in partitions:
    print(pt.name, ppt(rho_theo, pt))
    

A_BC -0.11749999999999995
AB_C -0.20499999999999993
AC_B -0.11749999999999997


In [16]:
print("PPT 1")
seeds_ppts_1 = {}
seeds_ppts_2 = {}
for pt in partitions:
    seeds_ppts_1[pt] = ppt(rho_exp_024, pt)
    print(seeds_ppts_1[pt], pt.name)
print('-------------')
print("PPT 2")
for pt in partitions:
    seeds_ppts_2[pt] = ppt(rho_exp_531, pt)
    print(seeds_ppts_2[pt], pt.name)
print("----------")

for pt in partitions:
    print("********")
    print("Partition:", pt.name)

    print("Copy 1:")
    _ppts_1 = np.array([ppt(r, pt) for r in rhos_single_copy_024_summed])
    _ppt_1_stat = aggregated_result.from_array(_ppts_1)
    _ppt_1_stat.display_in_notebook_q(seeds_ppts_1[pt])

    print("Copy 2:")
    _ppts_2 = np.array([ppt(r, pt) for r in rhos_single_copy_531_summed])
    _ppt_2_stat = aggregated_result.from_array(_ppts_2)
    _ppt_2_stat.display_in_notebook_q(seeds_ppts_2[pt])



PPT 1
-0.09447936273961244 A_BC
-0.17089033662588535 AB_C
-0.09524950306591506 AC_B
-------------
PPT 2
-0.10321785140938713 A_BC
-0.17670345251245614 AB_C
-0.09422616450628417 AC_B
----------
********
Partition: A_BC
Copy 1:


<IPython.core.display.Math object>

```latex
$(-94^{ +2}_{-3 }) \cdot 10^{ -3 }$
```

Copy 2:


<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

```latex
$(-95^{ +2}_{-4 }) \cdot 10^{ -3 }$
```

Copy 2:


<IPython.core.display.Math object>

```latex
$(-94^{ +3}_{-3 }) \cdot 10^{ -3 }$
```

see witnessing-two-copy-gme.html for single-copy witness values